In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
from __future__ import print_function

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score
from torchsummary import summary

In [4]:
import import_ipynb
from datagen import Data

importing Jupyter notebook from datagen.ipynb


In [5]:

seglength = 120
train_ratio = 0.8
focal_train, focal_test, focal_train_labels, focal_test_labels = Data('D:\EEG\FOCAL', train_ratio, seglength, class_num = 1)
ige_train, ige_test, ige_train_labels, ige_test_labels = Data('D:\EEG\IGE', train_ratio, seglength, class_num = 2)


In [6]:

val_ratio = 0.1
X_train = torch.cat((focal_train, ige_train))
Y_train = torch.cat((focal_train_labels, ige_train_labels))

X_test = torch.cat((focal_train, ige_train))
Y_test = torch.cat((focal_train_labels, ige_train_labels))

X_val = X_train[:int(val_ratio * len(Y_train))]
Y_val = Y_train[:int(val_ratio * len(Y_train))]

X_train = X_train[int(val_ratio * len(Y_train)):]
Y_train = Y_train[int(val_ratio * len(Y_train)):].view(-1, 1)




In [7]:
del focal_train, focal_test, focal_train_labels, focal_test_labels, ige_train, ige_test, ige_train_labels, ige_test_labels

In [8]:
np.shape(Y_train)

torch.Size([565143, 1])

In [36]:
def evaluate(model, X, Y, params = ["acc"]):
    results = []
    
    
    predicted = []
    """    
    batch_size=4096
    for i in range(int((len(X)/batch_size))):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = Variable(X[s:e])
        pred = model(inputs)
        
        predicted.append(pred.data.cpu().numpy())
    """        
        
    inputs = Variable(X)
    predicted = model(inputs)
    
    predicted = predicted.data.cpu().numpy()
    
    for param in params:
        if param == 'acc':
            results.append(accuracy_score(Y, np.round(predicted)))
        if param == "auc":
            results.append(roc_auc_score(Y, predicted))
        if param == "recall":
            results.append(recall_score(Y, np.round(predicted)))
        if param == "precision":
            results.append(precision_score(Y, np.round(predicted)))
        if param == "fmeasure":
            precision = precision_score(Y, np.round(predicted))
            recall = recall_score(Y, np.round(predicted))
            results.append(2*precision*recall/ (precision+recall))
    return results


In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.T = 9000

        # Layer 1
        self.conv1 = nn.Conv2d(1, 16, (1, 20))
        self.batchnorm1 = nn.BatchNorm2d(16, False)

        # Layer 2
        self.conv2 = nn.Conv2d(16, 32, (5, 1), stride=5)
        self.batchnorm2 = nn.BatchNorm2d(32, False)

        # FC
        self.fc1 = nn.Linear(32*24, 1)

    def forward(self, x):
        # Layer 1
        x = F.relu(self.conv1(x))
        x = self.batchnorm1(x)
        x = F.dropout(x, 0.25)

        # Layer 2
        x = F.relu(self.conv2(x))
        x = self.batchnorm2(x)


        x = x.permute(0, 3, 1, 2)



        x = x.reshape(-1, x.shape[1]* x.shape[2]* x.shape[3])
        x = F.sigmoid(self.fc1(x))
        return x



model = CNN()        

#model.forward(X_train[0:200])
#summary(model, (1, 120, 20))
#model.cpu().forward(X_train[0:2])

In [35]:
batch_size = 4096
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)

for epoch in range(10):  # loop over the dataset multiple times
    print("\nEpoch ", epoch)
    running_loss = 0.0
    for i in range(int(len(X_train)/batch_size-1)):
        s = i*batch_size
        e = i*batch_size+batch_size
        
        inputs = X_train[s:e]
        labels = Y_train[s:e]



        #inputs, labels = Variable(inputs.cuda(0)), Variable(labels.cuda(0))

        optimizer.zero_grad()
        outputs = model.forward(inputs)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        #print(loss.data)
    
    print(evaluate(model, X_train, Y_train, params=["fmeasure"]))


Epoch  0
[0.41929316929841365]

Epoch  1
[0.41929316929841365]

Epoch  2
[0.41929316929841365]

Epoch  3
[0.41929316929841365]

Epoch  4
[0.41929316929841365]

Epoch  5
[0.41929316929841365]

Epoch  6
[0.41929316929841365]

Epoch  7
[0.41929316929841365]

Epoch  8
[0.41929316929841365]

Epoch  9
[0.41929316929841365]
